In [1]:
import pandas as pd
import spacy
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import syllable_tokenize
import re
from spacy.tokens import Doc

In [2]:
df = pd.read_csv('merge_pathum_plan.csv')

In [3]:
df.head()

,StrategicIssues,project_name,PriorityOrder,Sub-activities,Strategy,ProjectIndicators,ImplementingAgency,AlignmentwithThailandVision,Budget,Unnamed: 9
0,ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให...,การพัฒนาบุคลากรเพื่อเพิ่มขีดความสามารถบริหารจั...,90.0,อบรมให้ความรู้ด้านการบริหารจัดการขยะและของเสีย...,กลยุทธ์ที่ 3 ส่งเสริมการจัดการขยะอย่างเป็นระบบ...,บุคลากรเข้าร่วมอบรมอย่างน้อย 800 คน,สำนักงานแรงงานจังหวัดปทุมธานี,3.3การจัดการทรัพยากรธรรมชาติและสิ่งแวดล้อม,706400.0,NaN
1,ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให...,การส่งเสริมการจัดการขยะในครัวเรือนส่วนราชการสถ...,7.0,1) จัดทําขอบเขตงาน\n2) กําหนดหลักเกณฑ์คุณสมบัต...,กลยุทธ์ที่ 3 ส่งเสริมการจัดการขยะอย่างเป็นระบบ...,ชุมชน ส่วนราชการสถานศึกษาศาสนสถาน มีส่วนร่วมใน...,สำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดป...,3.3การจัดการทรัพยากรธรรมชาติและสิ่งแวดล้อม,4323500.0,NaN
2,ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให...,ศูนย์การเรียนรู้การจัดการขยะและการผลิตขยะเชื้อ...,35.0,1) สํารวจสภาพพื้นที่และปริมาณขยะขององค์กรปกครอ...,กลยุทธ์ที่ 3 ส่งเสริมการจัดการขยะอย่างเป็นระบบ...,1) มีศูนย์การเรียนรู้การจัดการขยะและการสร้างมู...,สำนักงานพัฒนาพลังงานจังหวัดปทุมธานี,3.3การจัดการทรัพยากรธรรมชาติและสิ่งแวดล้อม,18000000.0,NaN
3,ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให...,ลดต้นทุนด้านพลังงานในสถานประกอบการในจังหวัดปทุ...,46.0,1) จัดอบรมสัมมนาผู้ประกอบการเอสเอ็มอีในจังหวัด...,กลยุทธ์ที่ 3 ส่งเสริมการจัดการขยะอย่างเป็นระบบ...,ผู้ประกอบการสามารถลดการใช้พลังงานได้ร้อยละ 10,สำนักงานพลังงานจังหวัดปทุมธานี,3.3การจัดการทรัพยากรธรรมชาติและสิ่งแวดล้อม,7094400.0,NaN
4,ประเด็นการพัฒนาที่ 1 : พัฒนาสภาพแวดล้อมเมืองให...,จัดซื้อตู้บรรทุกขยะมูลฝอยสถานีขนถ่ายขยะมูลฝอยต...,86.0,จัดซื้อตู้บรรทุกขยะมูลฝอยจํานวน 10 ตู้,กลยุทธ์ที่ 3 ส่งเสริมการจัดการขยะอย่างเป็นระบบ...,1. มีสภาพแวดล้อมที่ดีขึ้น\n2.ประชากรมีสุขภาพแล...,สำนักงานทรัพยากรธรรมชาติและสิ่งแวดล้อมจังหวัดป...,3.3การจัดการทรัพยากรธรรมชาติและสิ่งแวดล้อม,10000000.0,NaN


In [13]:
text = df['project_name'].tolist()

In [14]:
text

['การพัฒนาบุคลากรเพื่อเพิ่มขีดความสามารถบริหารจัดการขยะในสถานประกอบการ',
 'การส่งเสริมการจัดการขยะในครัวเรือนส่วนราชการสถานศึกษา',
 'ศูนย์การเรียนรู้การจัดการขยะและการผลิตขยะเชื้อเพลิง()',
 'ลดต้นทุนด้านพลังงานในสถานประกอบการในจังหวัดปทุมธานี',
 'จัดซื้อตู้บรรทุกขยะมูลฝอยสถานีขนถ่ายขยะมูลฝอยตะวันออกซอยรังสิตนครนายก17จํานวน10ตู้',
 'ส่งเสริมการจัดการน้ำเสียในชุมชน',
 'เก็บรวบรวมผักตบชวาเพื่อสร้างมูลค่าเพิ่มโดยการมีส่วนร่วม',
 'ฟื้นฟูระบบนิเวศน์แหล่งน้ำและทรัพยากรสัตว์น้ำในแม่น้ำเจ้าพระยาและคลองเชื่อมโยง',
 'ศึกษาวิจัยบําบัดน้ำเสียและการจัดการกลิ่น',
 'สร้างสระน้ำบริเวณวัดเนกขัมมาราม',
 'จัดหาเครื่องกังหันน้ำเติมอากาศเพื่อบําบัดน้ำเสียในตําบลคลองสาม',
 'ติดตั้งเครื่องบําบัดน้ำเสียพลังงานแสงอาทิตย์',
 'ส่งเสริมอุตสาหกรรมสีเขียว()',
 '(1)โครงการรณรงค์คัดแยกขยะ',
 '(2)คลองสวยน้ำใส',
 '(3)น้ำใสไร้มลพิษ',
 'พัฒนาระบบผลิตพืชแบบครบวงจร',
 'พัฒนามาตรฐานการผลิตสัตว์น้ำ',
 'การผลิตข้าวโภชนาการสูงแบบครบวงจรเพื่อเพิ่มมูลค่า',
 'การเพิ่มประสิทธิภาพการปรับปรุงเมล็ดพันธุ์ข้าว',
 'ส่งเสริมการผลิตสินค้าเ

In [15]:
# Define custom stopwords
custom_stopwords =["ไว้","ไม่","ไป","ได้","ให้","ใน","โดย","แห่ง","แล้ว","และ","แรก","แบบ","แต่","เอง","เห็น","เลย","เริ่ม","เรา","เมื่อ","เพื่อ","เพราะ","เป็นการ","เป็น","เปิดเผย","เปิด","เนื่องจาก","เดียวกัน","เดียว",
    "เช่น","เฉพาะ","เคย","เข้า","เขา","อีก","อาจ","อะไร","ออก","อย่าง","อยู่","อยาก","หาก","หลาย","หลังจาก","หลัง","หรือ","หนึ่ง","ส่วน","ส่ง","สุด","สําหรับ","ว่า","วัน","ลง","ร่วม","ราย","รับ","ระหว่าง","รวม","ยัง","มี",
    "มาก","มา","พร้อม","พบ","ผ่าน","ผล","บาง","น่า","นี้","นํา","นั้น","นัก","นอกจาก","ทุก","ที่สุด","ที่","ทําให้","ทํา","ทาง","ทั้งนี้","ถ้า","ถูก","ถึง","ต้อง","ต่างๆ","ต่าง","ต่อ","ตาม","ตั้งแต่","ตั้ง","ด้าน","ด้วย","ดัง","ซึ่ง",
    "ช่วง","จึง","จาก","จัด","จะ","คือ","ความ","ครั้ง","คง","ขึ้น","ของ","ขอ","ขณะ","ก่อน","ก็","การ","กับ","กัน","กว่า","กล่าว"]

def process_text(text, custom_stopwords):
    if pd.isna(text):  # Handle NaN values
        return []
    tokens = word_tokenize(text, engine="newmm")
    return [word for word in tokens if word not in custom_stopwords]

# Apply the function to the column
df['filtered_project_name'] = df['project_name'].apply(lambda x: process_text(x, custom_stopwords))

# Print results
token = df['filtered_project_name']

In [ ]:
text

In [ ]:
pj_name = []  # Initialize the list to store words
for word, tag in pos_tags:
    if tag == 'NOUN':  
        pj_name.append(word.strip())  # Append the word after stripping any extra spaces (if needed)

In [ ]:
pj_name

['',
 'ระบบ',
 'พืช',
 'แบบ',
 'ครบวงจร',
 '',
 '',
 'มาตรฐาน',
 'การผลิต',
 'สัตว์น้ำ',
 '',
 'การผลิต',
 'ข้าว',
 'โภชนาการ',
 'แบบ',
 'ครบวงจร',
 'มูลค่า',
 '',
 'การปรับปรุง',
 'เมล็ดพันธุ์',
 'ข้าว',
 '',
 'การผลิต',
 'สินค้า',
 'เกษตร',
 'แบบ',
 'ฟาร์ม',
 'อัจฉริยะ',
 '',
 'ภัยแล้ง',
 'เกษตรกร',
 'เทคโนโลยีพลังงาน',
 'ระบบ',
 'สูบน้ำ',
 'พลังงานแสงอาทิตย์',
 '',
 'บรรจุภัณฑ์',
 'ผลิตภัณฑ์',
 'OTOP',
 'ผลิตภัณฑ์',
 'ทาง',
 'การเกษตร',
 '',
 'มาตรฐาน',
 'การผลิต',
 'อาหาร',
 'นวัตกรรม',
 '',
 'ขีดความสามารถ',
 'การแข่งขัน',
 'อุตสาหกรรม',
 'แปรรูป',
 'การเกษตร',
 'อาหาร',
 '',
 'รณรงค์',
 'อาหาร',
 'ปลอดภัย',
 '',
 'ความปลอดภัย',
 'อาหาร',
 '',
 'การบริหาร',
 'สินค้า',
 'เกษตร',
 'ปลอดภัย',
 'กระบวนการ',
 'กลุ่ม',
 '',
 'มูลค่าเพิ่ม',
 'ตลาด',
 'สินค้า',
 'สุขภาพ',
 'ความงาม',
 '(Herbal&Nuceutical',
 'Market',
 '',
 'ความเชื่อมั่น',
 'ผู้บริโภค',
 'ตลาด',
 'การค้า',
 'ดิจิตอล',
 '',
 'คุณภาพ',
 'สินค้า',
 'มาตรฐาน',
 '',
 'แก้ไขปัญหา',
 'สาร',
 'เนื้อ',
 'แดง',
 'สุกร',
 '',
 'โรงง

In [ ]:
df['project_name']

0                            พัฒนาระบบผลิตพืชแบบครบวงจร
1                           พัฒนามาตรฐานการผลิตสัตว์น้ำ
2      การผลิตข้าวโภชนาการสูงแบบครบวงจรเพื่อเพิ่มมูลค่า
3         การเพิ่มประสิทธิภาพการปรับปรุงเมล็ดพันธุ์ข้าว
4            ส่งเสริมการผลิตสินค้าเกษตรแบบฟาร์มอัจฉริยะ
5     การแก้ภัยแล้งของเกษตรกรโดยใช้เทคโนโลยีพลังงานท...
6     พัฒนาบรรจุภัณฑ์ผลิตภัณฑ์ OTOP และผลิตภัณฑ์ทางก...
7      การยกระดับมาตรฐานการผลิตอาหารปลอดภัยด้วยนวัตกรรม
8     พัฒนาขีดความสามารถในการแข่งขันของอุตสาหกรรมแปร...
9                   รณรงค์ส่งเสริมการบริโภคอาหารปลอดภัย
10                           ส่งเสริมความปลอดภัยในอาหาร
11    พัฒนาการบริหารจัดการสินค้าเกษตรปลอดภัยผ่านกระบ...
12    สร้างมูลค่าเพิ่มตลาดสินค้าเกษตรเพื่อสุขภาพและค...
13    การสร้างความเชื่อมั่นผู้บริโภคสู่ตลาดการค้าดิจ...
14                     พัฒนาคุณภาพสินค้าเกษตรสู่มาตรฐาน
15                แก้ไขปัญหาการใช้สารเร่งเนื้อแดงในสุกร
16                ตรวจสอบโรงงานวัตถุอันตรายด้านปศุสัตว์
17                  พัฒนาเกษตรกรปราดเปรื่องด้านป

In [ ]:
# ฟังก์ชันกรองคำที่ไม่จำเป็น
def clean_text(text):

    text = text.strip()

    # ลบรูปแบบที่มีคำบ่งบอกจำนวนตามด้วยตัวเลขและคำต่อท้าย เช่น "จำนวน 1 ระบบ" หรือ "ระยะทาง 1.105 กม."
    text = re.sub(r'(จำนวน|ระยะทาง|ระยะห่าง|ไม่น้อยกว่า|ไม่ต่ำกว่า|รวม|พื้นที่|พื้นที่ไม่น้อยกว่า|ความยาวไม่น้อยกว่า|ครั้งละ|ขนาด|กว้าง|ยาว|สูง|หนา|เฉลี่ย|x|วงเงิน|รหัสสินทรัพย์|SERIAL NO\.|ครั้ง|ครั้งที่|หมายเลขทะเบียน|กต|ที่|สมัยที่|ม้วนละ|จำนวนเงิน)\s*[\d,]+(\.\d+)?\s*(คัน|ระบบ|กม\.|ตร\.ม\.|ซม\.|มม\.|กิโลเมตร|ML|ซีซี|เมตร|รายการ|ชิ้น|กระบอก|บาท|เล่ม|ผืน|ปทุมธานี|ประตู|นิ้ว|เส้น|BTUH|ชุด|เควี|แอมป์|ม้วน|ตารางเมตร|ถุง|ที่น่ัง|งาน|วัน|แห่ง|หลัง|ต้น|ตัว|เครื่อง|ชั่วโมง|บีทียู|เที่ยว|ขนาด|ขวด|หลอด)?', '', text)

    # ลบรูปแบบที่มีขนาดที่ใช้ x เป็นตัวคูณ เช่น "20x20x30"
    #text = re.sub(r'\b\d+(\.\d+)?(x\d+(\.\d+)?)+\b', '', text)

    # ลบเฉพาะ "- บ้าน", "- บ.", "- เขต", "- ที่อยู่" โดยไม่ลบข้อความอื่นๆ
    #text = re.sub(r'\s*-\s*(บ้าน|บ\.|เขต|ที่อยู่|คลอง|เลียบคลอง).*', '', text)

    # # ลบข้อความในวงเล็บที่มีคำบ่งบอกจำนวน เช่น (ตอนที่...), (หมายเลข...), (เลขครุภัณฑ์...)
    #text = re.sub(r'\((ตอนที่|หมายเลข|เลขครุภัณฑ์)[^\)]*\)', '', text)

    # ลบข้อความที่เกี่ยวกับปีงบประมาณและเดือน/ปี เช่น "ประจำปีงบประมาณ พ.ศ.2561", "ประจำเดือนตุลาคม 2560"
    text = re.sub(r"(ปี|ประจำปี|ประจำปีงบประมาณ|ประจำเดือน|ปีการศึกษา)\s.*$", "", text)

    # ลบตัวเลขทะเบียนที่มี "-" เช่น "82-7049"
    #text = re.sub(r'\b\d{2}-\d{4}\b', '', text).strip()
    #text = re.sub(r'\b\d+-\d+\b', '', text)
    #text = re.sub(r'\b\d-\d\b-\d\b', '', text)

    # # ลบข้อมูลที่ขึ้นต้นด้วยคำว่า "หมู่ที่" หรือ "เลขที่" ตามด้วยตัวเลข เช่น "หมู่ที่ 3,12,13"
    text = re.sub(r'\b(หมู่ที่|เลขที่)\s*[\d,]+\b', '', text)

    # # ลบข้อความที่เกี่ยวกับวัน, เดือน, ปี เช่น ศุกร์, 2 กันยายน
    text = re.sub(r'(วัน\s*(จันทร์|อังคาร|พุธ|พฤหัสบดี|ศุกร์|เสาร์|อาทิตย์))|(\d{1,2}\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม))|(\d{1,2}\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม)\s*\d{4})', '', text)

    # # ลบข้อความที่เกี่ยวกับวัน, เดือน, ปี
    text = re.sub(r'(วันที่|วันที่\s*\d+\s*(กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม|มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน)\s*\d{4}|พ.ศ\.\s*\d{4}|ประจำปี\s*งบประมาณ\s*\d{4}|ในวัน\s*(วันศุกร์|วันเสาร์|วันอาทิตย์|วันจันทร์|วันอังคาร|วันพุธ|วันพฤหัสบดี)|\s*\d+\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน)\s*\d{4}|ครั้งที่\s*\d+\s*ประจำปี\s*\d{4})', '', text)

    # # ลบตัวเลขและเดือนที่อยู่ติดกัน เช่น "2 กรกฎาคม"
    text = re.sub(r'\b\d{1,2}\s*(มกราคม|กุมภาพันธ์|มีนาคม|เมษายน|พฤษภาคม|มิถุนายน|กรกฎาคม|สิงหาคม|กันยายน|ตุลาคม|พฤศจิกายน|ธันวาคม)\b', '', text)

    # # ลบข้อความที่เกี่ยวกับที่อยู่ เช่น หมู่ที่, ตำบล, อำเภอ, จังหวัด
    text = re.sub(r'(หมู่ที่\s*\d+|ตำบล\s*[^ ]+|อำเภอ\s*[^ ]+|จังหวัด\s*[^ ]+|ซอย\s*[^ ]+|สมัย\s*[^ ]+|ช่วง\s*[^ ])', '', text)

    # # ลบข้อความเกี่ยวกับยี่ห้อและรุ่น เช่น "ยี่ห้อ" หรือ "รุ่น"
    text = re.sub(r'(ยี่ห้อ\s*[^ ]+|รุ่น\s*[^ ]+)', '', text)

    # ลบข้อความที่เกี่ยวกับหมายเลขทะเบียน, ยี่ห้อ, รหัสสินทรัพย์
    text = re.sub(r'(หมายเลขทะเบียน|ทะเบียน|ยี่ห้อ|รุ่น|หมู่|ปีที่|ฉบับที่|รหัสสินทรัพย์|หมายเลขครุภัณฑ์)\s*[^ ]+', '', text)

    # ลบทะเบียนรถรูปแบบ "กฉ 2767" หรือ "กม 1234"
    text = re.sub(r'\b[ก-ฮ]{1,2}\s?\d{4}\b', '', text)

    # ลบข้อความที่อยู่ในรูปแบบ "(93.14.18.00)" หรือคล้ายกัน
    text = re.sub(r'\(\d{2}\.\d{2}\.\d{2}\.\d{2}\)', '', text)

    # # ลบข้อความที่เกี่ยวกับหมายเลขทะเบียน (มีรูปแบบตัวเลขหรือรหัส)
    text = re.sub(r'\d{1,4}-\d{1,4}|\d{1,4} [A-Z]{1,2}-\d{1,4}|\([A-Za-z0-9\-]+\)', '', text)

    # # ลบตัวเลขที่อยู่ในวงเล็บเฉพาะที่มีตัวเลขภายในวงเล็บ
    text = re.sub(r'\(\d[\d\s]*\)', '', text)

    # # ลบตัวเลขที่มี 4 หลักขึ้นไป
    text = re.sub(r'\b\d{4,}\b', '', text)

    # # ลบตัวเลขโดด
    text = re.sub(r'\b\d\b', '', text)

    # # ลบตัวเลขทศนิยม เช่น 2.5, 100.55
    text = re.sub(r'\d+\.\d+', '', text)
    text = re.sub(r'\(\d+\.\d+\)', '', text)

    # # ลบตัวเลขที่มีเครื่องหมาย - เช่น 12-15, 100-200
    text = re.sub(r'\d+-\d+', '', text)
    text = re.sub(r'\(\d+-\d+\)', '', text)

    # # ลบตัวเลขที่มีเครื่องหมายคอมม่า เช่น 2,700
    text = re.sub(r'\d{1,3}(?:,\d{3})*', '', text)

    # ลบหน่วยที่เกี่ยวข้องกับตัวเลข เช่น เมตร, ตารางเมตร
    text = re.sub(r'\d+\s*(รายการ|เมตร|ตารางเมตร|กิโลกรัม|กิโลเมตร|ซีซี|ลิตร|ชิ้น|คัน|ชั่วโมง|วัน|ปี|เดือน|ครั้ง|รอบ|ลบ\.ม\.|ม\.)', '', text)

    #ข้างหน้าจำนวนข้างหลังเป็นอะไรก้ได้
    text = re.sub(r'(จำนวน).*', "", text)

    #ลบโดยวิธี ด้วยวิธี ข้างหลังเป็นอะไรก็ได้
    text = re.sub(r'(โดยวิธี|ด้วยวิธี).*', "", text)

    # ตัดคำจากข้อความ
    #words = word_tokenize(text, engine="newmm")

    # ลบวงเล็บที่ว่างหรือวงเล็บที่ไม่มีเนื้อหา
    text = re.sub(r"[()\[\]{}]", "", text)

    #ลบทุกช่องว่าง
    text = re.sub(r"\s+", "", text)

    #ลบลูกน้ำ
    text = re.sub(r",", "", text)

    #ลบภาษาอังกฤษ
    text = re.sub(r"^[a-zA-Z].*$", "", text)

    #ลบภาษาอังกฤษที่ผสมในประโยค
    text = re.sub(r"[a-zA-Z]", "", text)

    # ลบคำที่ขึ้นต้นด้วยตัวเลขตามด้วย "รายการ"
    text = re.sub(r"\d+รายการ", "", text)

    # ลบข้อความหลังคำว่า "สาย"
    text = re.sub(r"สาย.*$", "", text)

    # ลบคำว่า "ราคาจ้าง" และ "ราคาซื้อ"
    text = re.sub(r"ราคาจ้าง|ราคาซื้อ|ก่อสร้าง|จ้างก่อสร้าง", "", text)

    # ลบข้อความที่ระบุ ต. อ. และจังหวัด
    #text = re.sub(r"ต\..*?", "", text)  # ลบตั้งแต่ ต. ตามด้วยอะไรก็ได้
    #text = re.sub(r"อ\..*?", "", text)
    #text = re.sub(r"จ\..*?", "", text)

     # ลบคำว่า "การ" ที่เป็นคำเดี่ยว ๆ โดยไม่กระทบคำอื่น
    text =  re.sub(r"\bการ\b", "", text).strip()

    # รวมคำที่เหลือกลับเป็นข้อความ
    return text.strip()

# ใช้ฟังก์ชันกับคอลัมน์ 'project_name'

df['project_name'] = df['project_name'].apply(clean_text) #regex หลังตัดพวกจ้างเหมาออก

In [ ]:
df.head()

,StrategicIssues,project_name,PriorityOrder,Sub-activities,Strategy,ProjectIndicators,ImplementingAgency,AlignmentwithThailandVision,Budget,filtered_project_name
0,ประเด็นยุทธศาสตร์ที่ 2 : เสริมสร้างความเข้มแข็...,พัฒนาระบบผลิตพืชแบบครบวงจร,16.0,1. พัฒนาระบบการผลิตกล้วยน้ำว้าแบบครบวงจร ตามหล...,กลยุทธ์ที่ 1 ส่งเสริมบริโภคปลอดภัย,ร้อยละของจํานวนแปลงฟาร์มที่ผ่านการประเมินความพ...,สำนักงานเกษตรจังหวัดปทุมธานี,ด้านการสร้างความสามารถในการแข่งขัน,7800000.0,"[พัฒนา, ระบบ, ผลิต, พืช, ครบวงจร]"
1,ประเด็นยุทธศาสตร์ที่ 2 : เสริมสร้างความเข้มแข็...,พัฒนามาตรฐานการผลิตสัตว์น้ำ,54.0,1.อบรมหลักสูตร การจัดการฟาร์มเพาะเลี้ยงสัตว์น้...,กลยุทธ์ที่ 1 ส่งเสริมบริโภคปลอดภัย,1) ร้อยละเกษตรกรที่เข้าร่วมโครงการ ผ่านการเตรี...,สำนักงานประมงจังหวัดปทุมธานี,ด้านการสร้างความสามารถในการแข่งขัน,5500000.0,"[พัฒนา, มาตรฐาน, การผลิต, สัตว์น้ำ]"
2,ประเด็นยุทธศาสตร์ที่ 2 : เสริมสร้างความเข้มแข็...,การผลิตข้าวโภชนาการสูงแบบครบวงจรเพื่อเพิ่มมูลค่า,2.0,1) ศึกษาและรวมรวมข้อมูลพันธุ์ข้าวสี(โภชนาการสู...,กลยุทธ์ที่ 1 ส่งเสริมบริโภคปลอดภัย,1) ชาวนาสามารถผลิตข้าวสี (โภชนาการสูง) ได้คุณภ...,สำนักงานเกษตรและสหกรณ์จังหวัดปทุมธานี/ศูนย์วิจ...,ด้านการสร้างความสามารถในการแข่งขัน,4040000.0,"[การผลิต, ข้าว, โภชนาการ, สูง, ครบวงจร, เพิ่ม,..."
3,ประเด็นยุทธศาสตร์ที่ 2 : เสริมสร้างความเข้มแข็...,การเพิ่มประสิทธิภาพการปรับปรุงเมล็ดพันธุ์ข้าว,36.0,1. สํารวจเกษตรกร สมาชิกสหกรณ์ที่ผลิตเมล็ดพันธุ...,กลยุทธ์ที่ 1 ส่งเสริมบริโภคปลอดภัย,1.ร้อยละของต้นทุนการผลิตของเกษตรกรที่เข้าร่วมโ...,สำนักงานสหกรณ์จังหวัดปทุมธานี,ด้านการสร้างความสามารถในการแข่งขัน,10000000.0,"[เพิ่มประสิทธิภาพ, การปรับปรุง, เมล็ดพันธุ์, ข..."
4,ประเด็นยุทธศาสตร์ที่ 2 : เสริมสร้างความเข้มแข็...,ส่งเสริมการผลิตสินค้าเกษตรแบบฟาร์มอัจฉริยะ,10.0,1. การพัฒนาฐานข้อมูลเชิงพื้นที่และข้อมูลเปิดเพ...,กลยุทธ์ที่ 1 ส่งเสริมบริโภคปลอดภัย,ร้อยละของจํานวนแปลงฟาร์มที่ผ่านการประเมินความพ...,สำนักงานเกษตรและสหกรณ์จังหวัดปทุมธานี/ม.ธรรมศา...,ด้านการสร้างความสามารถในการแข่งขัน,9200000.0,"[ส่งเสริม, การผลิต, สินค้า, เกษตร, ฟาร์ม, อัจฉ..."


In [ ]:
df['project_name'].to_csv("viewp.csv")